In [56]:
import chromadb
from langchain.embeddings import OpenAIEmbeddings
from langchain.embeddings import SentenceTransformerEmbeddings

persistent_client = chromadb.PersistentClient()


collection = persistent_client.get_or_create_collection(
    "all-MiniLM-L6-v2_1000_split_clean")
display(collection.count())
embedding_model = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

2833

In [ ]:
collection.peek()

In [57]:
# Create context string

from langchain.chat_models import ChatOpenAI
topic = 'high rise development'

query = embedding_model.embed_documents(
    [topic],
)

southwark_results = collection.query(
    query_embeddings=query,
    where={"LPA": "London_Borough_of_Southwark"},
    n_results=10,
)

southwark_results_string = '\n\n'.join(southwark_results['documents'][0][0:5])

th_results = collection.query(
    query_embeddings=query,
    where={"LPA": "London_Borough_of_Tower_Hamlets"},
    n_results=10,
)
th_results_string = '\n\n'.join(th_results['documents'][0][0:5])


islington_results = collection.query(
    query_embeddings=query,
    where={"LPA": "London_Borough_of_Islington"},
    n_results=10,
)
islington_results_string = '\n\n'.join(islington_results['documents'][0][0:5])


context = '''
Context from Southwark: 
{southwark_results}

Context from Tower Hamlets: 
{th_results}

Context from Islington: 
{islington_results}


'''.format(southwark_results=southwark_results_string,
           th_results=th_results_string, islington_results=islington_results_string)

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=1)

query_string = '''
Compare the way Southwark, Tower Hamlets and Islington approach to the topic of {topic}, listing the three biggest differences. 

{context} 
'''. format(context=context, topic=topic)

result = llm.call_as_llm(query_string)

print('-------')

print(result)

-------
1. Approach to Design and Impact Assessment:
In Southwark, there is a strong emphasis on detailed modelling and analysis to assess the impacts of tall buildings. They prioritize well-designed buildings that respond sensitively to the local context and provide benefits for the community. On the other hand, Tower Hamlets focuses on maintaining the character and identity of an area. They require adequate distance between tall buildings and the preservation of existing local character. Islington also emphasizes the evaluation of benefits and justifications for tall buildings in relation to the surrounding context and local townscape.

2. Energy Efficiency and Sustainability:
Southwark's approach includes a focus on minimizing carbon emissions and reducing energy demand through energy-efficient design and low-carbon energy supply. They also encourage on-site renewable energy generation. Tower Hamlets does not explicitly mention energy efficiency and sustainability in the context pro